2. การทำนายการยกเลิกบริการของลูกค้า (Customer Churn Prediction)
โจทย์ Binary Classification ยอดนิยมอีกข้อที่เน้นการหา "สัญญาณ" ที่ซ่อนอยู่ในพฤติกรรมของลูกค้าก่อนที่พวกเขาจะจากไป

🎯 โจทย์: บริษัทโทรคมนาคมต้องการลดอัตราการยกเลิกบริการ (Churn) ของลูกค้า โดยมีข้อมูลโปรไฟล์ลูกค้า, ประเภทของบริการที่ใช้ (อินเทอร์เน็ต, สตรีมมิ่ง, โทรศัพท์), และข้อมูลสัญญา/การจ่ายเงิน เพื่อสร้างโมเดลทำนายว่าลูกค้าคนใดมีแนวโน้มจะ "ยกเลิกบริการ" (Target=1) ในไตรมาสถัดไป

In [1]:
# ติดตั้ง AutoGluon และ library สำหรับดาวน์โหลดข้อมูลจาก Kaggle
# ขั้นตอนนี้อาจใช้เวลา 2-3 นาที
%pip install --upgrade "autogluon[all]" opendatasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━

In [2]:
import opendatasets as od
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

# ระบุ URL ของชุดข้อมูลบน Kaggle
dataset_url = 'https://www.kaggle.com/datasets/blastchar/telco-customer-churn'

# ดาวน์โหลดข้อมูล (ระบบจะถามหา Kaggle Username และ Key)
od.download(dataset_url)

# กำหนด Path ของไฟล์ข้อมูล
data_path = '/content/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv'

# โหลดข้อมูลด้วย pandas
df = pd.read_csv(data_path)

# แสดงขนาดและตัวอย่างข้อมูลเพื่อตรวจสอบความถูกต้อง
print(f"ขนาดของข้อมูล: {df.shape}")
print("\nตัวอย่างข้อมูล 5 แถวแรก:")
df.head()


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: surasan092
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/blastchar/telco-customer-churn


100%|██████████| 172k/172k [00:00<00:00, 225MB/s]


ขนาดของข้อมูล: (7043, 21)

ตัวอย่างข้อมูล 5 แถวแรก:


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
# ทำความสะอาดข้อมูลเบื้องต้น (ถ้าจำเป็น)
# TotalCharges ควรเป็นตัวเลข แต่มีบางค่าเป็นช่องว่าง ทำให้ Type เป็น object
# เราจะแปลงให้เป็นตัวเลข และแทนที่ค่าที่ผิดพลาดด้วยค่าว่าง (NaN)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# แปลงคอลัมน์เป้าหมาย 'Churn' เป็น 0 และ 1
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

# แบ่งข้อมูลเป็น train (80%) และ test (20%) เพื่อจำลองการแข่งขัน
# random_state=42 เพื่อให้สามารถทำซ้ำผลลัพธ์เดิมได้ทุกครั้ง
train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)

# เก็บ customerID ของ test_data ไว้สำหรับสร้างไฟล์ submission
test_customer_ids = test_data['customerID']
# ลบคอลัมน์คำตอบ (Churn) ออกจาก test_data เพราะในสถานการณ์จริงเราจะไม่รู้ค่านี้
test_data_no_label = test_data.drop(columns=['Churn'])


print(f"ขนาดข้อมูลสำหรับ Train: {train_data.shape}")
print(f"ขนาดข้อมูลสำหรับ Test:  {test_data_no_label.shape}")


ขนาดข้อมูลสำหรับ Train: (5634, 21)
ขนาดข้อมูลสำหรับ Test:  (1409, 20)


In [4]:
# กำหนดคอลัมน์เป้าหมาย
label = 'Churn'

# กำหนดเวลาในการฝึก (หน่วยเป็นวินาที), 300 วินาที = 5 นาที
# สำหรับโจทย์นี้ ข้อมูลไม่ซับซ้อนมาก ใช้เวลาไม่นานก็ได้โมเดลที่ดี
time_limit = 300

# สร้าง Predictor
predictor = TabularPredictor(
    label=label,
    eval_metric='f1',
    path='ag_models_churn' # โฟลเดอร์สำหรับเซฟโมเดล
)

# เริ่มฝึกโมเดล
# AutoGluon จะจัดการ Missing Values และ Categorical Features ให้เอง
predictor.fit(
    train_data,
    time_limit=time_limit,
    presets='best_quality'
)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       11.45 GB / 12.67 GB (90.3%)
Disk Space Avail:   65.02 GB / 107.72 GB (60.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stack

In [5]:
# แสดง Leaderboard ของโมเดลทั้งหมด
# โมเดลที่มี score_val สูงที่สุดคือโมเดลที่ดีที่สุด
# สังเกตว่า AutoGluon ได้ลองโมเดลหลากหลายประเภท ตั้งแต่ LightGBM, CatBoost ไปจนถึง Neural Network
predictor.leaderboard(train_data, silent=True)


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr_BAG_L1,0.990482,0.568468,f1,0.285247,0.348592,2.415980,0.285247,0.348592,2.415980,1,True,6
1,RandomForestGini_BAG_L1,0.988536,0.566629,f1,0.318692,0.362328,2.769221,0.318692,0.362328,2.769221,1,True,5
2,KNeighborsDist_BAG_L1,0.977502,0.512638,f1,0.039563,0.029413,0.010487,0.039563,0.029413,0.010487,1,True,2
3,LightGBM_BAG_L1,0.766851,0.605139,f1,0.787520,0.208121,42.946868,0.787520,0.208121,42.946868,1,True,4
4,LightGBMXT_BAG_L1,0.738326,0.610047,f1,4.504517,0.344496,43.501087,4.504517,0.344496,43.501087,1,True,3
5,WeightedEnsemble_L2,0.738326,0.610047,f1,4.507135,0.351236,44.313843,0.002618,0.006740,0.812756,2,True,8
6,KNeighborsUnif_BAG_L1,0.680122,0.518791,f1,0.041788,0.026695,0.008761,0.041788,0.026695,0.008761,1,True,1
7,CatBoost_BAG_L1,0.669424,0.590103,f1,0.142570,0.217068,72.114210,0.142570,0.217068,72.114210,1,True,7


In [6]:
# ทำนายผลบนข้อมูล test
predictions = predictor.predict(test_data_no_label)

# สร้าง DataFrame สำหรับไฟล์ submission
submission = pd.DataFrame({
    'customerID': test_customer_ids,
    'Churn': predictions
})

# แสดงตัวอย่าง 5 แถวแรกของไฟล์ submission
print("ตัวอย่างไฟล์ Submission:")
submission.head()

# บันทึกเป็นไฟล์ CSV
submission.to_csv('submission_churn.csv', index=False)

print("\nสร้างไฟล์ submission_churn.csv เรียบร้อยแล้ว!")


ตัวอย่างไฟล์ Submission:

สร้างไฟล์ submission_churn.csv เรียบร้อยแล้ว!


In [8]:
submission['Churn'].value_counts()

,count
Churn,
0,969
1,440
